# Set up credentials

In [8]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION="6e85ef31-541f-4d92-bb9b-4c004bf0e1cd"
RESOURCE_GROUP="toan658033-rg"
WS_NAME="2024-06_Toan_MLOps"
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

# Verify credentials went through

In [9]:
# Verify that the handle works correctly.  
# If you ge an error here, modify your SUBSCRIPTION, RESOURCE_GROUP, and WS_NAME in the previous cell.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location,":", ws.resource_group)

southcentralus : toan658033-rg


# Create "src" folder

In [10]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

# Create main.py file in src folder

In [11]:
# %pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 3.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
%%writefile {train_src_dir}/main3.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
# from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    args = parser.parse_args()
   
    # Start Logging
    mlflow.start_run()

    # enable autologging
    mlflow.sklearn.autolog()

    ###################
    #<prepare the data>
    ###################
    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("input data:", args.data)
    
    credit_df = pd.read_csv(args.data, header=1, index_col=0)

    mlflow.log_metric("num_samples", credit_df.shape[0])
    mlflow.log_metric("num_features", credit_df.shape[1] - 1)

    train_df, test_df = train_test_split(
        credit_df,
        test_size=args.test_train_ratio,
        random_seed=42
    )
    ####################
    #</prepare the data>
    ####################
    
    ##################
    #<train the model>
    ##################
    # Extracting the label column
    y_train = train_df.pop("default payment next month")

    # convert the dataframe values to array
    X_train = train_df.values

    # Extracting the label column
    y_test = test_df.pop("default payment next month")

    # convert the dataframe values to array
    X_test = test_df.values

    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    X_test_smote, y_test_smote = smote.fit_resample(X_test, y_test)

    train_data = lgb.Dataset(X_train_smote, label=y_train_smote)
    test_data = lgb.Dataset(X_test_smote, label=y_test_smote, reference=train_data)

    print(f"Training with data of shape {X_train.shape}")

    params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
    }


    clf = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', num_leaves=31, learning_rate=0.05, n_estimators=100)
    clf.fit(X_train_smote, y_train_smote, eval_set=[(X_test_smote, y_test_smote)], eval_metric='logloss', early_stopping_rounds=10)

    # Make predictions
    y_pred = clf.predict(X_test_smote)

    print(classification_report(y_test_smote, y_pred))
    ###################
    #</train the model>
    ###################

    ##########################
    #<save and register model>
    ##########################
    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=clf,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=clf,
        path=os.path.join(args.registered_model_name, "trained_model"),
    )
    ###########################
    #</save and register model>
    ###########################
    
    # Stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

Writing ./src/main3.py


# Set up job for credit defaults model

In [13]:
from azure.ai.ml import command
from azure.ai.ml import Input

registered_model_name = "credit_defaults_model_lgbm"

job = command(
    inputs=dict(
        data=Input(
            type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/credit_card/default_of_credit_card_clients.csv",
        ),
        test_train_ratio=0.2,
        learning_rate=0.25,
        registered_model_name=registered_model_name,
    ),
    code="./src/",  # location of source code
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    display_name="credit_default_prediction_lgbm",
)

# Create job for credit defaults prediction

In [14]:
ml_client.create_or_update(job)

Uploading src (0.01 MBs): 100%|██████████| 10086/10086 [00:00<00:00, 58771.21it/s]




Experiment,Name,Type,Status,Details Page
toan658033,wheat_rhythm_jpwxhv3ttg,command,Starting,Link to Azure Machine Learning studio
